In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Karratha,AU,2021-04-03 21:39:59,-20.7377,116.8463,81.30,44,33,6.49
1,1,Saint-Pierre,RE,2021-04-03 21:39:59,-21.3393,55.4781,77.00,78,0,6.91
2,2,Hastings,GB,2021-04-03 21:37:45,50.8552,0.5729,44.60,65,90,18.41
3,3,Bayeux,BR,2021-04-03 21:39:59,-7.1250,-34.9322,78.80,94,75,11.50
4,4,Suratgarh,IN,2021-04-03 21:40:00,29.3167,73.9000,72.45,8,19,9.78


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
temps

NameError: name 'temps' is not defined

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()

# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
    
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps)

# Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)


In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 30
What is the maximum temperature you would like for your trip? 70


In [12]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Hastings,GB,2021-04-03 21:37:45,50.8552,0.5729,44.60,65,90,18.41
6,6,Ushuaia,AR,2021-04-03 21:40:00,-54.8000,-68.3000,39.20,81,40,20.71
9,9,Mangai,CD,2021-04-03 21:40:01,-4.0500,19.5333,69.89,96,100,0.51
10,10,Ribeira Grande,PT,2021-04-03 21:40:01,38.5167,-28.7000,62.26,83,25,29.01
11,11,Busselton,AU,2021-04-03 21:40:03,-33.6500,115.3333,62.01,93,100,7.00
13,13,Oranjemund,NaN,2021-04-03 21:40:04,-28.5500,16.4333,61.93,85,0,8.10
14,14,Katsuura,JP,2021-04-03 21:40:04,35.1333,140.3000,64.00,80,71,3.00
15,15,Strezhevoy,RU,2021-04-03 21:40:04,60.7333,77.5889,35.60,87,0,8.95
16,16,Albany,US,2021-04-03 21:36:18,42.6001,-73.9662,51.01,21,20,8.05
17,17,Plouzane,FR,2021-04-03 21:40:05,48.3833,-4.6167,42.80,75,0,12.66


In [13]:
preferred_cities_df.count()

City_ID       271
City          271
Country       270
Date          271
Lat           271
Lng           271
Max Temp      271
Humidity      271
Cloudiness    271
Wind Speed    271
dtype: int64

In [14]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Hastings,GB,44.60,50.8552,0.5729,
6,Ushuaia,AR,39.20,-54.8000,-68.3000,
9,Mangai,CD,69.89,-4.0500,19.5333,
10,Ribeira Grande,PT,62.26,38.5167,-28.7000,
11,Busselton,AU,62.01,-33.6500,115.3333,
13,Oranjemund,NaN,61.93,-28.5500,16.4333,
14,Katsuura,JP,64.00,35.1333,140.3000,
15,Strezhevoy,RU,35.60,60.7333,77.5889,
16,Albany,US,51.01,42.6001,-73.9662,
17,Plouzane,FR,42.80,48.3833,-4.6167,


In [15]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [16]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
   # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [23]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [24]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))